In [16]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import random
from torch.nn import functional as F

model_name = "flax-community/papuGaPT2"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device

device(type='cuda')

In [ ]:
class Trie:
    is_end: bool
    children: dict[str, "Trie"]

    def __init__(self):
        self.is_end = False
        self.children = {}

    def search(self, s):
        node = self
        for ch in s:
            if ch not in node.children:
                return None
            node = node.children[ch]
        return node if node.is_end else None

    def search_by_letter(self, s, letter):
        node = self.search(letter)
        if node is None:
            return None
        return node.search(s)

    def insert(self, s):
        node = self
        for ch in s:
            if ch not in node.children:
                node.children[ch] = Trie()
            node = node.children[ch]
        node.is_end = True

    def print_all_nodes(self, prefix=""):
        if self.is_end:
            print(prefix)
        for head, child in self.children.items():
            child.print_all_nodes(prefix + head)

In [17]:
# Inicjalizacja drzewa trie
trie = Trie()
tokens = tokenizer.get_vocab().keys()
current_word = ""
for token in tokens:
    if token.startswith("Ġ"):  # Sprawdzenie, czy token zaczyna nowe słowo
        if current_word:
            trie.insert(current_word)
        current_word = token[1:]  # Usuń prefiks "Ġ"
    else:
        current_word += token

# Dodanie ostatniego słowa
if current_word:
    trie.insert(current_word)

# Wypisanie wszystkich słów w drzewie trie
trie.print_all_nodes()

su
sungObie
such
suche
suchejPiÄĻkna
suchego
suchawce
suchÄħ
sucholendersÅĤa
suchy
suchych
suchym
suk
sukniiÄĻkumatycznie
sukniadesa
suknie
sukie
sukien
sukienka
sukienkiė
sukienkÄĻ
sukienek
sukce
sukcesieliwÄħ
sukcesy
sukcesywniemour
sukcesem
sukcesMinisterons
sukcesÃ³w
sukcesuniemy
suszar
suszarkiZnaleÎ»
suszarka
suszenia
suszymus
suszone
suszonych
sushi
sum
sumadakty
sumysimy
sumie
sumieniem
sumienia
sumÄĻ
suple
suplementÃ³w
suplementlansuwaÅ¼ne
suplementu
suplementy
super
superboha
supermarke
supDro
sur
surowekÅĤadanie
surowego
surowi
surowy
surowymOdno22brandKorwickiÅ¼ar
surowych
surowce
surowcÃ³w
surowca
surowoCiedowÄħ
surf
sub
subtelny
subtelnedziach
subtenowskierzadgodnia
substancjami
substancjakowspÃ³ÅĤmowaÄĩsznieÃ·
substancjÄĻ
substancjijacego
substancjeczÄĻsplay
substanjnymi
subskryp
subskrypcji
subiektyw
subwoo
suge
sugestie
suger
sugerowaÄĩ
sugerujeDodatkÎ¹
sugerujÄħ
suwaÅ¼szym
suwak
suweren
sufi
sufit
sufitu
se
ser
serw
serwi
serwis
serwisowe
serwisowych
serwisach
serwise

In [19]:
def best_k(prefix, K=10):
    input_ids = tokenizer(prefix, return_tensors="pt")["input_ids"].to(device)
    output = model(input_ids=input_ids)
    next_token_logits = output.logits[0, -1, :]
    probs = F.softmax(next_token_logits, dim=-1)
    d = {}
    current_word = prefix

    for i in range(probs.shape[0]):
        token = tokenizer.decode([i])
        if token.startswith("Ġ"):  # Nowe słowo
            if current_word:
                trie.insert(current_word)
            current_word = token[1:]  # Usuń prefiks "Ġ"
        else:
            current_word += token
        d[token] = probs[i]

    # Dodaj ostatnie słowo do drzewa trie
    if current_word:
        trie.insert(current_word)
    return [
        (t, d[t])
        for t in sorted(d, key=d.get, reverse=True)[:K]
        if trie.search_by_letter(t, prefix[0])
    ]


def sample_from_pairs(pairs):
    tokens = [p[0] for p in pairs]
    weights = [p[1] for p in pairs]
    return random.choices(tokens, weights=weights, k=1)[0]


def sample_demo(N, txt):
    for i in range(N):
        d = best_k(txt)
        print(txt)
        next_token = sample_from_pairs(d)
        for t, p in best_k(txt):
            star = ""
            if t == next_token:
                star = "*"
            print(f"   [{t}]{star} {p:.4f}")
        txt += next_token
        print()


sample_demo(3, "do dnia dzisiejszego")

do dnia dzisiejszego


IndexError: list index out of range